# GINI Calculation

Gini impurity is a measure of misclassification, which applies in a multiclass classifier context.

Gini coefficient applies to binary classification and requires a classifier that can in some way rank examples according to the likelihood of being in a positive class.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy as sci

surrogate convex loss --> minimize classification error

cost complexity pruning --> minimize gini impurity

In [2]:
def error(p):
    return 1 - np.max([p, 1-p])

In [3]:
X_train, X_test, y_train, y_test = pd.read_pickle('./saved_variables/X_y_train_test')
df_X, df_y, df_data = pd.read_pickle('saved_variables/df_X_y_data')
feature_names, class_names = pd.read_pickle('saved_variables/feature_class_names')
selected_features, df_X_sel, X_train_sel, X_test_sel = pd.read_pickle('saved_variables/selected')

## work on entropy!!

In [21]:
def entropy(dist):
    su=0
    for p in dist:
        r= p/sum(dist)
        if r==0:
            su+=0
        else:
            su+= -r*(np.log(r))
    return su/np.log(2)

In [5]:

#def entropy4(labels, base=None):
#    value,counts = np.unique(labels, return_counts=True)
#    norm_counts = counts / counts.sum()
#    base = e if base is None else base
#    return -(norm_counts * np.log(norm_counts)/np.log(base)).sum()


In [6]:
#def entropy(n, labels):
#    ent = 0
#    for label in labels.keys():
#        p_x = labels[label] / n
#        ent += - p_x * math.log(p_x, 2)
#    return ent

In [7]:
list = [9,5]

In [8]:
entropy(list)

0.940285958670631

In [9]:
34

34

## end of entropy

In [16]:
def GRLC(values):
    '''
    Calculate Gini index, Gini coefficient, Robin Hood index, and points of 
    Lorenz curve based on the instructions given in 
    www.peterrosenmai.com/lorenz-curve-graphing-tool-and-gini-coefficient-calculator
    Lorenz curve values as given as lists of x & y points [[x1, x2], [y1, y2]]
    @param values: List of values
    @return: [Gini index, Gini coefficient, Robin Hood index, [Lorenz curve]] 
    '''
    n = len(values)
    assert(n > 0), 'Empty list of values'
    sortedValues = sorted(values) #Sort smallest to largest

    #Find cumulative totals
    cumm = [0]
    for i in range(n):
        cumm.append(sum(sortedValues[0:(i + 1)]))

    #Calculate Lorenz points
    LorenzPoints = [[], []]
    sumYs = 0           #Some of all y values
    robinHoodIdx = -1   #Robin Hood index max(x_i, y_i)
    for i in range(1, n + 2):
        x = 100.0 * (i - 1)/n
        y = 100.0 * (cumm[i - 1]/float(cumm[n]))
        LorenzPoints[0].append(x)
        LorenzPoints[1].append(y)
        sumYs += y
        maxX_Y = x - y
        if maxX_Y > robinHoodIdx: robinHoodIdx = maxX_Y   
    
    giniIdx = 100 + (100 - 2 * sumYs)/n #Gini index 
    
    def entropy(dist):
        su=0
        for p in dist:
            r= p/sum(dist)
            if r==0:
                su+=0
            else:
                su+= -r*(np.log(r))
        return su/np.log(2)
    
    

    return [giniIdx, giniIdx/100, robinHoodIdx, LorenzPoints]

In [11]:
# selected features is calculated prior to person deciding if they will use them or not
df_gini = pd.DataFrame(columns=['feature_names', 'gini_index', 'gini_coeffiecient', 'robin_hood_index',
                                'lorenz_curve_x', 'lorenz_curve_y'])
df_gini_sel = pd.DataFrame(columns=['feature_names', 'gini_index', 'gini_coeffiecient', 'robin_hood_index',
                                    'lorenz_curve_x', 'lorenz_curve_y'])

df_gs, df_Xs, sfns= [df_gini, df_gini_sel], [df_X, df_X_sel], [feature_names, selected_features]

for i in range(2):

    gini_index_list = []
    gini_coeffiecient_list = []
    robin_hood_index_list = []
    lorenz_curve_x_list = []
    lorenz_curve_y_list = []

    column_index = range(len(df_Xs[i].columns))

    for f in column_index:

        result = GRLC(df_Xs[i][df_Xs[i].columns[f]])

        gini_index_list.append(result[0])
        gini_coeffiecient_list.append(result[1])
        robin_hood_index_list.append(result[2])
        lorenz_curve_x_list.append(result[3][0])
        lorenz_curve_y_list.append(result[3][1])
    df_gs[i]['feature_names'] = sfns[i]
    df_gs[i]['gini_index'] = gini_index_list
    df_gs[i]['gini_coeffiecient'] = gini_coeffiecient_list
    df_gs[i]['robin_hood_index'] = robin_hood_index_list
    df_gs[i]['lorenz_curve_x'] = lorenz_curve_x_list
    df_gs[i]['lorenz_curve_y'] = lorenz_curve_y_list
    
# output: df_gini, df_gini_sel

In [33]:
df_gini

,feature_names,gini_index,gini_coeffiecient,robin_hood_index,lorenz_curve_x,lorenz_curve_y
0,radius_mean,13.596051,0.135961,9.739615,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.08684532761314426, 0.1825232268643536,..."
1,texture_mean,12.390332,0.123903,8.774044,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.08846727485260768, 0.18303888277949415..."
2,perimeter_mean,14.358725,0.143587,10.347275,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.08367988155255132, 0.17525192822983518..."
3,area_mean,27.591985,0.275920,20.116641,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.03850985382625588, 0.08423862798649283..."
4,smoothness_mean,8.136095,0.081361,5.791281,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.095989348702329, 0.20999835853289303, ..."
5,compactness_mean,27.238183,0.272382,19.697552,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.03264273786668759, 0.07212394403774831..."
6,concavity_mean,47.118843,0.471188,35.223185,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,concave_points_mean,42.482572,0.424826,32.154895,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,symmetry_mean,8.305296,0.083053,5.836151,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.10283165391133776, 0.21604348420806527..."
9,fractal_dimension_mean,5.989612,0.059896,4.225044,"[0.0, 0.1757469244288225, 0.351493848857645, 0...","[0.0, 0.13981927602944602, 0.28042216689652705..."


In [41]:
for i in range(2,4):
    print(i)

2
3


# violin plot

Robin Hood or Schutz index

The Robin Hood index is equivalent to the maximum vertical distance between the Lorenz curve, or the cumulative portion of the total income held below a certain income percentile, and the Perfect Equality Line, that is the 45 degree line of equal incomes.

The value of the index approximates the share of total income that needs to be transferred from households above the mean to those below the mean to achieve equality in the distribution of incomes.

http://www.maxi-pedia.com/Robin+Hood+index

Entopy
https://medium.com/activating-robotic-minds/demystifying-entropy-f2c3221e2550

In [43]:

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

df_gs, df_Xs, sfns= [df_gini, df_gini_sel], [df_X, df_X_sel], [feature_names, selected_features]

pointpos_male = [-0.9,-1.1,-0.6,-0.3]
pointpos_female = [0.45,0.55,1,0.4]
show_legend = [True,False,False,False]

fig = go.Figure()

for i in range(2,4):
    fig.add_trace(go.Violin(x=df_gini[df_gini.columns[i]],
                            y=df_gini[df_gini.columns[i]],
                            legendgroup='M', scalegroup='M', name='M',
                            side='negative',
                            pointpos=pointpos_male[i], # where to position points
                            line_color='blue',
                            showlegend=show_legend[i])
             )
    fig.add_trace(go.Violin(x=df_gini_sel[df_gini.columns[i]],
                            y=df_gini_sel[df_gini.columns[i]],
                            legendgroup='After Permuation Importance', scalegroup='F', name='F',
                            side='positive',
                            pointpos=pointpos_female[i],
                            line_color='yellow',
                            showlegend=show_legend[i])
             )

# update characteristics shared by all traces
fig.update_traces(meanline_visible=True,
                  points='all', # show all points
                  jitter=0.05,  # add some jitter on points for better visibility
                  scalemode='count') #scale violin plot area with total count
fig.update_layout(
    title_text="Total bill distribution<br><i>scaled by number of bills per gender",
    violingap=0, violingroupgap=0, violinmode='overlay')
fig.show()

In [46]:
x=df['day']
print(x)

0       Sun
1       Sun
2       Sun
3       Sun
4       Sun
5       Sun
6       Sun
7       Sun
8       Sun
9       Sun
10      Sun
11      Sun
12      Sun
13      Sun
14      Sun
15      Sun
16      Sun
17      Sun
18      Sun
19      Sat
20      Sat
21      Sat
22      Sat
23      Sat
24      Sat
25      Sat
26      Sat
27      Sat
28      Sat
29      Sat
       ... 
214     Sat
215     Sat
216     Sat
217     Sat
218     Sat
219     Sat
220     Fri
221     Fri
222     Fri
223     Fri
224     Fri
225     Fri
226     Fri
227     Sat
228     Sat
229     Sat
230     Sat
231     Sat
232     Sat
233     Sat
234     Sat
235     Sat
236     Sat
237     Sat
238     Sat
239     Sat
240     Sat
241     Sat
242     Sat
243    Thur
Name: day, Length: 244, dtype: object


In [34]:
df_gini['gini_coeffiecient']

0     0.135961
1     0.123903
2     0.143587
3     0.275920
4     0.081361
5     0.272382
6     0.471188
7     0.424826
8     0.083053
9     0.059896
10    0.319634
11    0.236860
12    0.320416
13    0.441318
14    0.212140
15    0.354219
16    0.410860
17    0.274836
18    0.199359
19    0.308149
20    0.159872
21    0.134466
22    0.168482
23    0.322375
24    0.096285
25    0.325292
26    0.414828
27    0.324102
28    0.111382
29    0.112039
Name: gini_coeffiecient, dtype: float64

In [35]:
x=df['day'][(df['sex'] == 'Male') & (df['day'] == pd.unique(df['day'])[i])]
print(x)
y=df['total_bill'][(df['sex'] == 'Male')& (df['day'] == pd.unique(df['day'])[i])]
print(y)

90     Fri
91     Fri
95     Fri
96     Fri
97     Fri
98     Fri
99     Fri
220    Fri
222    Fri
224    Fri
Name: day, dtype: object
90     28.97
91     22.49
95     40.17
96     27.28
97     12.03
98     21.01
99     12.46
220    12.16
222     8.58
224    13.42
Name: total_bill, dtype: float64


In [ ]:

#in creating a function that auto generates traces?
categorical_dimensions = ['body-style', 'drive-wheels', 'fuel-type']

dimensions = [dict(values=cars_df[label], label=label) for label in categorical_dimensions]


# Lorenz Curve

In [47]:
import chart_studio.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np

N = 500
random_x = np.linspace(0, 1, N)
random_y = np.random.randn(N)

# Create a trace
trace = go.Scatter(
    x = random_x,
    y = random_y
)

data = [trace]

py.iplot(data, filename='basic-line')

PlotlyRequestError: Authentication credentials were not provided.

In [ ]:
d={}
for x in range(1,10):
        d["string{0}".format(x)]="Hello"

In [7]: d["string5"]
Out[7]: 'Hello'

In [ ]:
df_lorenz = pd.DataFrame(columns=['ind_feature_name', 'ind_lorenz_x', 'ind_lorenz_y'])
for coord in len(df_lorenz)

In [ ]:
#input:df_gini  output:fig.show()
import plotly.express as px

fig = px.line(df_gini, x="lorenz_curve_x", y="lorenz_curve_y", color='feature_names')
#x= cumunalative proportion of feature div by n
#y= 
fig.show()

In [ ]:
import chart_studio.plotly as py
from plotly.graph_objs import *
#py.sign_in('username', 'api_key')
trace1 = {
  "uid": "b1cb09", 
  "name": "_Tnaa____10_Temp55_Mn10_Tnaa20_Gly20_DMSO15_TagTime5", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.0106458481192335, 0.0227111426543648, 0.0361958836053939, 0.050390347764371894, 0.06529453513129879, 0.0801987224982257, 0.09581263307310149, 0.11213626685592619, 0.12845990063875087, 0.14549325762952448, 0.16323633782824698, 0.18097941802696949, 0.1994322214336409, 0.21788502484031227, 0.23633782824698368, 0.255500354861604, 0.27466288147622425, 0.2945351312987935, 0.3144073811213627, 0.33498935415188075, 0.35557132718239887, 0.37686302342086586, 0.39815471965933286, 0.41944641589779985, 0.4414478353442158, 0.46344925479063165, 0.48616039744499645, 0.5088715400993612, 0.5322924059616749, 0.5557132718239887, 0.5798438608942512, 0.6039744499645139, 0.6288147622427254, 0.6536550745209369, 0.6792051100070973, 0.7047551454932577, 0.7310149041873669, 0.7579843860894251, 0.7849538679914834, 0.8126330731014905, 0.8410220014194464, 0.8701206529453513, 0.8999290276792051, 0.9311568488289567, 0.9638041163946061, 1]
}
trace2 = {
  "uid": "9ba442", 
  "name": "_Tnaa____11_Temp55_Mn1_Tnaa20_Gly20_DMSO15_TagTime5", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.008233532934131737, 0.017964071856287425, 0.02844311377245509, 0.040419161676646706, 0.0531437125748503, 0.06661676646706587, 0.08008982035928144, 0.09431137724550898, 0.1092814371257485, 0.12425149700598802, 0.13997005988023953, 0.156437125748503, 0.17290419161676646, 0.19011976047904192, 0.20733532934131738, 0.2252994011976048, 0.2432634730538922, 0.2619760479041916, 0.280688622754491, 0.3001497005988024, 0.3203592814371258, 0.3405688622754491, 0.3615269461077844, 0.38248502994011974, 0.4041916167664671, 0.4258982035928144, 0.44835329341317365, 0.4708083832335329, 0.4940119760479042, 0.5179640718562875, 0.5419161676646707, 0.5666167664670658, 0.5920658682634731, 0.6175149700598802, 0.6437125748502994, 0.6706586826347305, 0.6983532934131736, 0.7267964071856288, 0.7559880239520959, 0.7859281437125748, 0.8166167664670658, 0.8488023952095808, 0.8824850299401198, 0.9176646706586826, 0.9558383233532934, 1]
}
trace3 = {
  "uid": "ff06ab", 
  "name": "_Tnaa____12_Temp55_Mn0025_Tnaa20_Gly20_DMSO15_TagTime5", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.005502063273727648, 0.013067400275103164, 0.021320495185694635, 0.030949105914718018, 0.04126547455295736, 0.052269601100412656, 0.0639614855570839, 0.07634112792297111, 0.08940852819807428, 0.1031636863823934, 0.11691884456671252, 0.1313617606602476, 0.14649243466299863, 0.1623108665749656, 0.1781292984869326, 0.19463548830811556, 0.21182943603851445, 0.2297111416781293, 0.24759284731774414, 0.266162310866575, 0.28541953232462175, 0.3046767537826685, 0.3246217331499312, 0.3452544704264099, 0.36657496561210456, 0.3878954607977992, 0.40990371389270974, 0.4325997248968363, 0.45598349381017883, 0.48005502063273725, 0.5048143053645117, 0.5302613480055021, 0.5563961485557084, 0.5832187070151307, 0.6107290233837689, 0.6389270976616231, 0.6678129298486932, 0.6973865199449794, 0.7283356258596974, 0.7606602475928473, 0.7943603851444292, 0.8294360385144429, 0.8665749656121046, 0.905777166437414, 0.9491059147180193, 1]
}
trace4 = {
  "uid": "ba2ace", 
  "name": "_Tnaa____60_Temp37_Mn0025_Tnaa20_Gly20_DMSO15_TagTime5", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.006293706293706294, 0.013986013986013986, 0.023076923076923078, 0.032867132867132866, 0.043356643356643354, 0.05454545454545454, 0.06643356643356643, 0.07902097902097902, 0.09160839160839161, 0.1048951048951049, 0.11888111888111888, 0.13356643356643358, 0.14825174825174825, 0.16363636363636364, 0.1797202797202797, 0.1958041958041958, 0.2125874125874126, 0.23006993006993007, 0.24755244755244754, 0.26573426573426573, 0.2846153846153846, 0.3034965034965035, 0.3230769230769231, 0.34335664335664334, 0.36363636363636365, 0.38461538461538464, 0.4062937062937063, 0.4286713286713287, 0.45174825174825173, 0.47482517482517483, 0.4986013986013986, 0.5230769230769231, 0.5482517482517483, 0.5741258741258741, 0.6006993006993007, 0.6286713286713287, 0.6573426573426573, 0.6867132867132867, 0.7174825174825175, 0.7496503496503496, 0.7832167832167832, 0.8188811188811189, 0.8566433566433567, 0.8972027972027972, 0.9426573426573427, 1]
}
trace5 = {
  "uid": "222c53", 
  "name": "_Tnaa____70_Temp37_Mn0025_Tnaa20_Gly20_DMSO15_TagTime10", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.002044989775051125, 0.006816632583503749, 0.013633265167007498, 0.021813224267211998, 0.03135650988411725, 0.04158145875937287, 0.05248807089297887, 0.06475800954328562, 0.07770961145194274, 0.09134287661895024, 0.1056578050443081, 0.12065439672801637, 0.136332651670075, 0.1520109066121336, 0.1683708248125426, 0.18541240627130198, 0.20313565098841171, 0.22154055896387184, 0.23994546693933197, 0.25903203817314246, 0.27880027266530333, 0.2992501704158146, 0.3203817314246762, 0.34151329243353784, 0.36332651670074984, 0.3858214042263122, 0.40899795501022496, 0.4328561690524881, 0.4567143830947512, 0.4812542603953647, 0.5064758009543285, 0.5323790047716428, 0.5589638718473074, 0.5862304021813224, 0.6141785957736878, 0.6428084526244036, 0.6721199727334697, 0.7027948193592365, 0.7341513292433538, 0.7668711656441718, 0.8002726653033402, 0.8350374914792092, 0.8718473074301295, 0.9107021131561008, 0.9522835719154737, 1]
}
trace6 = {
  "uid": "737d34", 
  "name": "_Tnaa____78_Temp37_Mn0025_Tnaa20_Gly20_DMSO15_TagTime10", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.0006802721088435374, 0.003401360544217687, 0.00816326530612245, 0.014965986394557823, 0.02312925170068027, 0.03197278911564626, 0.04217687074829932, 0.053061224489795916, 0.06462585034013606, 0.07755102040816327, 0.09115646258503401, 0.1054421768707483, 0.12040816326530612, 0.1360544217687075, 0.1523809523809524, 0.16938775510204082, 0.18639455782312925, 0.20408163265306123, 0.22244897959183674, 0.24149659863945577, 0.2612244897959184, 0.2816326530612245, 0.30272108843537415, 0.3238095238095238, 0.345578231292517, 0.36802721088435375, 0.391156462585034, 0.41496598639455784, 0.43945578231292515, 0.46462585034013604, 0.49047619047619045, 0.5170068027210885, 0.54421768707483, 0.572108843537415, 0.6006802721088436, 0.6299319727891156, 0.6605442176870748, 0.6918367346938775, 0.7238095238095238, 0.7571428571428571, 0.7918367346938775, 0.827891156462585, 0.8659863945578231, 0.9061224489795918, 0.9496598639455782, 1]
}
trace7 = {
  "uid": "87c595", 
  "name": "_Tnaa____86_Temp55_Mn0025_Tnaa20_Gly20_DMSO15_TagTime5", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.0006811989100817438, 0.0027247956403269754, 0.006130790190735695, 0.010899182561307902, 0.0170299727520436, 0.023841961852861037, 0.03201634877384196, 0.04087193460490463, 0.05040871934604905, 0.060626702997275204, 0.07152588555858311, 0.0837874659400545, 0.09673024523160763, 0.11035422343324251, 0.12465940054495912, 0.1396457765667575, 0.1553133514986376, 0.17166212534059946, 0.18869209809264306, 0.2064032697547684, 0.22479564032697547, 0.2438692098092643, 0.26362397820163486, 0.2840599455040872, 0.30517711171662126, 0.32697547683923706, 0.34945504087193463, 0.3726158038147139, 0.39645776566757496, 0.42166212534059944, 0.4475476839237057, 0.47411444141689374, 0.5013623978201635, 0.5299727520435967, 0.5592643051771117, 0.5899182561307902, 0.6219346049046321, 0.6546321525885559, 0.6886920980926431, 0.7247956403269755, 0.7622615803814714, 0.8017711171662125, 0.8440054495912807, 0.8896457765667575, 0.9393732970027248, 1]
}
trace8 = {
  "uid": "e0605e", 
  "name": "_Tnaa____94_Temp55_Mn10_Tnaa20_Gly20_DMSO15_TagTime5", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.011004126547455296, 0.024071526822558458, 0.03782668500687758, 0.052269601100412656, 0.06740027510316368, 0.08321870701513068, 0.09972489683631362, 0.11623108665749655, 0.13342503438789546, 0.15061898211829436, 0.16850068775790922, 0.18638239339752408, 0.20495185694635487, 0.2235213204951857, 0.24277854195323245, 0.2620357634112792, 0.28198074277854196, 0.30192572214580465, 0.3218707015130674, 0.3425034387895461, 0.3631361760660248, 0.3844566712517194, 0.405777166437414, 0.4270976616231087, 0.44910591471801925, 0.47111416781292986, 0.4938101788170564, 0.5165061898211829, 0.5392022008253095, 0.562585969738652, 0.5859697386519945, 0.610041265474553, 0.6341127922971114, 0.6588720770288858, 0.6836313617606602, 0.7090784044016506, 0.7352132049518569, 0.7613480055020633, 0.7881705639614855, 0.8156808803301238, 0.843878954607978, 0.8727647867950481, 0.9023383768913342, 0.9325997248968363, 0.9649243466299863, 1]
}
trace9 = {
  "uid": "84698a", 
  "name": "_Tnaa____97_06X_Cleanup", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.009569377990430622, 0.021189336978810664, 0.03349282296650718, 0.047163362952836636, 0.06151742993848257, 0.07655502392344497, 0.09159261790840738, 0.10731373889268626, 0.12371838687628162, 0.14080656185919344, 0.15789473684210525, 0.17566643882433355, 0.19343814080656185, 0.21189336978810663, 0.2303485987696514, 0.24948735475051265, 0.2686261107313739, 0.2884483937115516, 0.3082706766917293, 0.3287764866712235, 0.3492822966507177, 0.37047163362952834, 0.39166097060833904, 0.41353383458646614, 0.4354066985645933, 0.45727956254272045, 0.47983595352016406, 0.5023923444976076, 0.5256322624743678, 0.5488721804511278, 0.5727956254272044, 0.5974025974025974, 0.6220095693779905, 0.6473000683526999, 0.6725905673274094, 0.6985645933014354, 0.7252221462747779, 0.7525632262474368, 0.7799043062200957, 0.807928913192071, 0.836637047163363, 0.8660287081339713, 0.8967874231032126, 0.9289131920710868, 0.9624060150375939, 1]
}
trace10 = {
  "uid": "90ab8d", 
  "name": "_Tnaa____98_08X_Cleanup", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.009569377990430622, 0.021189336978810664, 0.03349282296650718, 0.047163362952836636, 0.06151742993848257, 0.07655502392344497, 0.09159261790840738, 0.10731373889268626, 0.12371838687628162, 0.14012303485987695, 0.1572112098427888, 0.1749829118250171, 0.19275461380724537, 0.21120984278879015, 0.22966507177033493, 0.24880382775119617, 0.2679425837320574, 0.2877648667122351, 0.30758714969241285, 0.32809295967190705, 0.34859876965140124, 0.3697881066302119, 0.39097744360902253, 0.41216678058783324, 0.43403964456596034, 0.4559125085440875, 0.4784688995215311, 0.5010252904989747, 0.5242652084757348, 0.5475051264524948, 0.5714285714285714, 0.5960355434039645, 0.6206425153793574, 0.645933014354067, 0.6712235133287765, 0.6971975393028025, 0.7238550922761449, 0.7511961722488039, 0.7785372522214627, 0.8065618591934381, 0.83526999316473, 0.8653451811346549, 0.8961038961038961, 0.9282296650717703, 0.9624060150375939, 1]
}
trace11 = {
  "uid": "5db99d", 
  "name": "_Tnaa____99_1ng_input_amped", 
  "x": [0, 0.02127659574468085, 0.0425531914893617, 0.06382978723404255, 0.0851063829787234, 0.10638297872340426, 0.1276595744680851, 0.14893617021276595, 0.1702127659574468, 0.19148936170212766, 0.2127659574468085, 0.23404255319148937, 0.2553191489361702, 0.2765957446808511, 0.2978723404255319, 0.3191489361702128, 0.3404255319148936, 0.3617021276595745, 0.3829787234042553, 0.40425531914893614, 0.425531914893617, 0.44680851063829785, 0.46808510638297873, 0.48936170212765956, 0.5106382978723404, 0.5319148936170213, 0.5531914893617021, 0.574468085106383, 0.5957446808510638, 0.6170212765957447, 0.6382978723404256, 0.6595744680851063, 0.6808510638297872, 0.7021276595744681, 0.723404255319149, 0.7446808510638298, 0.7659574468085106, 0.7872340425531915, 0.8085106382978723, 0.8297872340425532, 0.851063829787234, 0.8723404255319149, 0.8936170212765957, 0.9148936170212766, 0.9361702127659575, 0.9574468085106383, 0.9787234042553191], 
  "y": [0, 0.011821974965229486, 0.02573018080667594, 0.04033379694019471, 0.055632823365785816, 0.07162726008344923, 0.08762169680111266, 0.1043115438108484, 0.12169680111265646, 0.13908205841446453, 0.1571627260083449, 0.17524339360222532, 0.1933240611961057, 0.2121001390820584, 0.23087621696801114, 0.25034770514603616, 0.2698191933240612, 0.2899860917941586, 0.3101529902642559, 0.33031988873435325, 0.35118219749652296, 0.3720445062586926, 0.39290681502086233, 0.4144645340751043, 0.43602225312934634, 0.4575799721835883, 0.47983310152990266, 0.502086230876217, 0.5250347705146036, 0.5479833101529903, 0.5709318497913769, 0.5945757997218358, 0.6182197496522949, 0.6425591098748261, 0.6668984700973575, 0.691933240611961, 0.7169680111265647, 0.7426981919332406, 0.7684283727399166, 0.7948539638386648, 0.8219749652294854, 0.849095966620306, 0.8769123783031989, 0.9054242002781642, 0.935326842837274, 0.9666203059805285, 1]
}
data = Data([trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10, trace11])
layout = {
  "title": "<b>Lorenz Curve</b>", 
  "width": 1274, 
  "xaxis": {
    "type": "linear", 
    "range": [0, 1], 
    "title": "<b>Cumulative Fraction of Total Target</b>"
  }, 
  "yaxis": {
    "type": "linear", 
    "range": [0, 1], 
    "title": "<b>Cumulative Fraction per Base Depth</b>"
  }, 
  "height": 632, 
  "autosize": True
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig)